# Homework 12

https://scikit-learn.org/0.15/modules/scaling_strategies.html#incremental-learning

* Implement a mini batch functionality to train a regressor.
    - (Optional) If anyone want to do this in a pipeline can do this: https://koaning.github.io/tokenwiser/api/pipeline.html

* Save model, load the model again and test it on `X_test` __Do NOT commit the pickle file__

# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import joblib


from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [2]:
def test_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)

    df = df.sample(5000, random_state=100).reset_index(drop=True)
    
    y = df['sellingprice']
    df.drop('sellingprice', axis=1, inplace=True)
    X = df
    
    return X,y

def partial_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)
   
    while(True):
        yield df.sample(100).reset_index(drop=True)
        
gen = partial_df()

In [3]:
X_test, y_test = test_df()

In [4]:
# each time you call this you will get a new slice of the dataframe.
next(gen)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2006,Nissan,Sentra,1.8,Sedan,manual,3n1cb51d56l548001,mo,1.9,72997.0,black,gray,first community credit union,4325,1900,Tue Mar 10 2015 04:00:00 GMT-0700 (PDT)
1,2012,Acura,MDX,Technology Package,suv,NaN,2hnyd2h38ch540000,pa,4.4,43637.0,gray,black,royal coast auto inc,27300,27600,Fri Jun 05 2015 02:00:00 GMT-0700 (PDT)
2,2002,Mazda,Millenia,Premium,Sedan,automatic,jm1ta221421726159,wa,2.7,141118.0,silver,gray,subaru of spokane,1175,1300,Wed Jan 21 2015 04:30:00 GMT-0800 (PST)
3,2006,Mercedes-Benz,E-Class,E350 4MATIC,Sedan,automatic,wdbuf87j26x200132,nj,2.7,119323.0,black,beige,credit acceptance corp/vrs/southfield,7100,5000,Tue May 26 2015 03:45:00 GMT-0700 (PDT)
4,2013,Kia,Optima,SX,Sedan,automatic,5xxgr4a61dg164571,ca,3.4,28080.0,black,black,kia motors finance,20700,21500,Tue Jan 13 2015 12:30:00 GMT-0800 (PST)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2004,cadillac,srx,4x2 suv,NaN,automatic,1gyee63ax40151815,nc,3,124971.0,white,beige,north state acceptance,3225,2700,Tue Jan 06 2015 01:30:00 GMT-0800 (PST)
96,2013,Toyota,Corolla,L,sedan,automatic,5yfbu4ee3dp145828,fl,4.2,38787.0,gray,beige,santander consumer,10550,9800,Wed Jun 17 2015 02:05:00 GMT-0700 (PDT)
97,2011,Ford,F-150,King Ranch,supercrew,automatic,1ftfw1et7bkd34146,tx,4.1,87247.0,white,brown,rlb investments,24700,26200,Wed Jun 17 2015 03:20:00 GMT-0700 (PDT)
98,2013,Chevrolet,Cruze,2LT,sedan,automatic,1g1pe5sb7d7247520,wi,3.4,40422.0,silver,black,the hertz corporation,10900,10800,Wed Jun 03 2015 03:00:00 GMT-0700 (PDT)


---
# Preprocessing the dataset

In [5]:
def process_train(df):
    '''
    This function does preprocessing for train and test datasets.
    '''
    drop_cols = ['vin', 'saledate']
    df.drop(drop_cols, axis = 1, inplace = True)
    
    df = pd.get_dummies(data = df, columns = ['transmission'], drop_first = True)
    
    df[['year', 'make', 'model', 'trim', 'body', 'state','condition', 'odometer', 'color', 'interior',  'seller', 'mmr']] = df[['year', 'make', 'model', 'trim', 'body', 'state','condition', 'odometer', 'color', 'interior',  'seller', 'mmr']].apply(encoder.fit_transform)
    df.fillna(999, inplace = True)
    
    return df

# Building The Model

In [6]:
for i in range(18):
    new_df = next(gen)
    
    processed_df = process_train(new_df)
    X_train, y_train = processed_df[['year', 'make', 'model', 'trim', 'body', 'state', 'condition', 'odometer', 'color', 'interior', 'seller', 'mmr']], processed_df['sellingprice']
    
    sgd = SGDRegressor()
    sgd.partial_fit(X_train, y_train)

# Saving The Model

In [7]:
joblib.dump(sgd, 'sgd.pkl')

['sgd.pkl']

# Testing The Model

In [8]:
sgd = joblib.load('sgd.pkl')

In [9]:
processed_test = process_train(X_test)

In [10]:
X_test = processed_test[['year', 'make', 'model', 'trim', 'body', 'state', 'condition', 'odometer', 'color', 'interior', 'seller', 'mmr']]

In [11]:
test_preds = sgd.predict(X_test)

In [12]:
mean_squared_error(y_test, test_preds)

1.7906148343616736e+28

In [13]:
sgd

SGDRegressor()